In [ ]:
import date
import box_score
import helper_functions
import pandas as pd
import bets_scrape
import evaluator
import model

!sudo apt-get update
!sudo apt-get install chromium-driver -y

date = date.Date()
month, day = date.date_month_day(-1)

#extract yesterday's games statistics and add to cumulative statistics history 
box_score = box_score.BoxScore()
box_score_result = box_score.full_box_scores(str(month), str(day))
box_score_result.to_csv("/work/NBA-Bets-Box-Score-Results/" + "NBA-Bets-Box-Score-Results-" + month + "-" + day +".csv")
all_box_score_results = box_score.update_all_box_score_results(month, day)
all_box_score_results.to_csv("/work/All_Box_Score_Results_updated.csv")

#evaluate yesterday's predictions
predictions = pd.read_csv("/work/NBA-Bets-Predictions/" + "NBA-Bets-" + month + "-" + day +".csv")
box_score_result['name'] = box_score_result['name'].apply(helper_functions.abbrv)
evaluations = evaluator.Evaluator()
evaluated_predictions = evaluations.predictions_evaluator(predictions, box_score_result)
final_evaluation = evaluations.past_games_trends(evaluated_predictions, all_box_score_results, True)
final_evaluation.to_csv("/work/NBA-Bets-Evaluations/" + "NBA-Bets-Evaluations-" + month + "-" + day +".csv")

#evaluate yesterday's optimized predictions
optimized_predictions = pd.read_csv("/work/NBA-Bets-Optimized-Predictions/" + "NBA-Bets-" + month + "-" + day +".csv")
optimized_predictions['Correct'] = evaluations.optimized_predictions_evaluator(optimized_predictions, final_evaluation)

#add to cumulative evaluations history 
evaluations.all_optimizer_evaluations()
all_evaluations = evaluations.all_evaluations()
all_evaluations.to_csv("/work/All_Evaluations.csv")

#initialize model
model = model.Model()
all_evaluations = model.preprocessing(all_evaluations, True)

#scrape bets predictions
bets = bets_scrape.Bets()

#schedule to run 10 mins before first scheduled game today 
running_time = bets.schedule()
while True:
    today_date_formatted = date.date_formatting(0)
    if str(today_date_formatted.split(' ')[1]) == str(running_time):
        month, day = date.date_month_day(0)

        #extract today's bet predictions
        final_bets = bets.output(all_box_score_results)
        final_bets.to_csv('/work/NBA-Bets-Predictions/' + 'NBA-Bets-' + month + "-" + day + ".csv")
        bets_inputs = evaluations.past_games_trends(final_bets, all_box_score_results, False)

        #input today's predictions into model
        bets_inputs = model.preprocessing(bets_inputs, False)
        features = model.features_processing(list(all_evaluations.columns[:len(all_evaluations.columns) - 1])[5:])
        target = [all_evaluations.columns[-1]]

        #training model
        X, Y, final_model = model.train(features, target, all_evaluations)
        final_model.fit(X, Y)
        for i in set(features).difference(set(bets_inputs.columns[:len(bets_inputs.columns) - 1])):
            bets_inputs[i] = 0

        #model predicts to optimize from today's set of predictions
        X_predictions = bets_inputs.loc[:, features].values
        predictions = final_model.predict(X_predictions) 
        indices = [i for i in range(len(predictions)) if predictions[i] == 'Y']
        final_bets.iloc[indices, :].to_csv('/work/NBA-Bets-Optimized-Predictions/' + 'NBA-Optimized-Bets-' + month + "-" + day + ".csv")
        break

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=be013728-c0d9-4cb4-b2ed-943c2a6d561f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>